# GTEx model with different priors

## Load libraries

In [1]:
if (!requireNamespace("PLIER", quietly = TRUE)) {
    devtools::install_github("wgmao/PLIER")
}

# 3. Install PLIER2 (mchikina/PLIER2) if not already installed
if (!requireNamespace("PLIER2", quietly = TRUE)) {
    REPO_PATH <- "/home/msubirana/Documents/pivlab/PLIER2"  # adjust
    remotes::install_local(REPO_PATH, force = TRUE, dependencies = FALSE)
}

library(bigstatsr)
library(data.table)
library(dplyr)
library(rsvd)
library(glmnet)
library(Matrix)
library(knitr)
library(here)
library(PLIER2)
library(org.Hs.eg.db)
library(clusterProfiler)
library(httr)
library(tidyr)
library(AnnotationDbi)

source(here("config.R"))


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Matrix

Loaded glmnet 4.1-10

here() starts at /home/msubirana/Documents/pivlab/plier2-analyses


Attaching package: ‘PLIER2’


The following object is masked from ‘package:bigstatsr’:

    AUC


Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, 

## Output directory

In [2]:
output_data_dir <- config$GTEx$DATASET_FOLDER
dir.create(output_data_dir, showWarnings = FALSE, recursive = TRUE)

In [3]:
gtex_plier2_kegg <- readRDS(file.path(output_data_dir, "gtex_KEGG_2021_Human_PLIER2.rds"))

kegg_plier2_summary <- gtex_plier2_kegg$summary %>%
    dplyr::filter(FDR < 0.05 & AUC > 0.7) %>%
    dplyr::group_by(LV) %>%
    dplyr::summarise(pathway = paste(pathway, collapse = ', '), .groups = "drop") %>%
    dplyr::arrange(as.numeric(gsub("[^0-9]", "", LV)))

In [4]:
gtex_baseRes <- readRDS(file.path(output_data_dir, "gtex_PLIER2_baseRes.rds"))

In [5]:
gtex_genes <- readRDS(file.path(output_data_dir, "gtex_genes.rds"))

In [6]:
colnames(gtex_baseRes$Z) <- paste0('LV', seq_len(ncol(gtex_baseRes$Z)))
rownames(gtex_baseRes$Z) <- gtex_genes

In [7]:
head(gtex_baseRes$Z)

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,⋯,LV403,LV404,LV405,LV406,LV407,LV408,LV409,LV410,LV411,LV412
WASH7P,0,0,0.0000000,0,0,0.0000000,0,0,0,0,⋯,0.0000000,0.0000000,0.0000000,0,0,0.0000000,0.0000000,0,0.0000000,0.0000000
RP11-34P13.15,0,0,0.2197233,0,0,0.3177482,0,0,0,0,⋯,0.2191181,0.0000000,0.0000000,0,0,0.1963381,0.0000000,0,0.0000000,0.0000000
RP11-34P13.16,0,0,0.0000000,0,0,0.3203991,0,0,0,0,⋯,0.2079666,0.0000000,0.0000000,0,0,0.2021600,0.0000000,0,0.0000000,0.0000000
RP11-34P13.18,0,0,0.0000000,0,0,0.0000000,0,0,0,0,⋯,0.0000000,0.2309318,0.0000000,0,0,0.0000000,0.0000000,0,0.1516040,0.0000000
AP006222.2,0,0,0.0000000,0,0,0.0000000,0,0,0,0,⋯,0.0000000,0.0000000,0.0000000,0,0,0.0000000,0.1519592,0,0.1020665,0.2276423
MTND1P23,0,0,0.0000000,0,0,0.0000000,0,0,0,0,⋯,0.0000000,0.0000000,0.2902236,0,0,2.2067290,0.0000000,0,0.0000000,0.0000000


In [8]:
kegg_plier2_Z <- data.frame(gtex_baseRes$Z)

In [9]:
Z <- gtex_baseRes$Z

map <- bitr(rownames(Z), fromType = "SYMBOL", toType = "ENTREZID", OrgDb = org.Hs.eg.db)
map <- map[!duplicated(map$SYMBOL), ]

lv_names <- colnames(Z)

top_syms <- setNames(vector("list", length(lv_names)), lv_names)

'select()' returned 1:many mapping between keys and columns

Warning message in bitr(rownames(Z), fromType = "SYMBOL", toType = "ENTREZID", OrgDb = org.Hs.eg.db):
“20.4% of input gene IDs are fail to map...”


In [10]:
head(top_syms$LV1)

NULL

In [11]:
kegg_plier2_Z %>% 
dplyr::select(LV1)  %>% 
arrange(desc(LV1)) %>%
tail(10)

,LV1
,<dbl>
MT-ND4,0
MT-TH,0
MT-TS2,0
MT-TL2,0
MT-ND5,0
MT-ND6,0
MT-TE,0
MT-CYB,0
MT-TT,0


In [12]:
for (lv in lv_names) {
  v <- Z[, lv]
  idx <- which(!is.na(v) & v > 0)
  if (length(idx) == 0L) { top_syms[[lv]] <- character(0); next }
  ord <- idx[order(v[idx], decreasing = TRUE)]
  top_syms[[lv]] <- rownames(Z)[ord]
}

top_entrez <- lapply(top_syms, function(syms) {
  ids <- map$ENTREZID[match(syms, map$SYMBOL)]
  unique(as.character(ids[!is.na(ids)]))
})

kegg_list <- lapply(top_entrez, function(ids) {
  if (!length(ids)) return(NULL)
  enrichKEGG(gene = ids, organism = "hsa", qvalueCutoff = 0.05)
})

Reading KEGG annotation online: "https://rest.kegg.jp/link/hsa/pathway"...

Reading KEGG annotation online: "https://rest.kegg.jp/list/pathway/hsa"...



In [13]:
kegg_list$LV15@result  %>% 
dplyr::filter(p.adjust < 0.05)  %>% 
dplyr::pull(Description) 

[1] "Peroxisome"                                  
 [2] "Fatty acid degradation"                      
 [3] "Valine, leucine and isoleucine degradation"  
 [4] "Biosynthesis of cofactors"                   
 [5] "Carbon metabolism"                           
 [6] "Fatty acid metabolism"                       
 [7] "Glycine, serine and threonine metabolism"    
 [8] "PPAR signaling pathway"                      
 [9] "Alcoholic liver disease"                     
[10] "Tryptophan metabolism"                       
[11] "One carbon pool by folate"                   
[12] "Pyruvate metabolism"                         
[13] "beta-Alanine metabolism"                     
[14] "Complement and coagulation cascades"         
[15] "Biosynthesis of unsaturated fatty acids"     
[16] "Butanoate metabolism"                        
[17] "Propanoate metabolism"                       
[18] "Drug metabolism - cytochrome P450"           
[19] "Metabolism of xenobiotics by cytochrome P450"
[20] "Retinol metabolism"                          
[21] "Glyoxylate and dicarboxylate metabolism"     
[22] "Histidine metabolism"                        
[23] "Tyrosine metabolism"                         
[24] "Folate transport and metabolism"             
[25] "Pantothenate and CoA biosynthesis"           
[26] "Folate biosynthesis"                         
[27] "Biosynthesis of amino acids"                 
[28] "Glycolysis / Gluconeogenesis"                
[29] "Vitamin digestion and absorption"            
[30] "Arginine and proline metabolism"             
[31] "Nicotinate and nicotinamide metabolism"      
[32] "Cysteine and methionine metabolism"          
[33] "Chemical carcinogenesis - DNA adducts"       
[34] "Alanine, aspartate and glutamate metabolism" 
[35] "Glutathione metabolism"                      
[36] "Cholesterol metabolism"                      
[37] "Arginine biosynthesis"                       
[38] "Fatty acid elongation"                       
[39] "Selenocompound metabolism"                   
[40] "2-Oxocarboxylic acid metabolism"             
[41] "Ferroptosis"                                 
[42] "Drug metabolism - other enzymes"             
[43] "Citrate cycle (TCA cycle)"                   
[44] "Lysine degradation"                          
[45] "Terpenoid backbone biosynthesis"             
[46] "Glycerolipid metabolism"                     
[47] "Steroid biosynthesis"                        
[48] "Lipoic acid metabolism"                      
[49] "Biosynthesis of nucleotide sugars"           
[50] "Primary bile acid biosynthesis"              
[51] "Sulfur metabolism"                           
[52] "Insulin resistance"                          
[53] "Pentose phosphate pathway"                   
[54] "Amino sugar and nucleotide sugar metabolism" 
[55] "Platinum drug resistance"                    
[56] "Antifolate resistance"                       
[57] "ABC transporters"                            
[58] "Pertussis"                                   
[59] "Glucagon signaling pathway"                  
[60] "Ascorbate and aldarate metabolism"           
[61] "Glycerophospholipid metabolism"              
[62] "Pyrimidine metabolism"                       
[63] "AMPK signaling pathway"                      
[64] "Proximal tubule bicarbonate reclamation"     
[65] "Insulin signaling pathway"                   
[66] "Mineral absorption"                          
[67] "Nucleotide metabolism"

In [14]:
kegg_plier2_summary  %>%  dplyr::filter(LV == "LV15")

LV,pathway
<chr>,<chr>
LV15,"Drug metabolism, Glycine, serine and threonine metabolism, Peroxisome, Retinol metabolism"


In [15]:
kegg_list$LV102@result  %>% 
dplyr::filter(p.adjust < 0.05)  %>% 
dplyr::pull(Description) 

[1] "Osteoclast differentiation"                                   
  [2] "Leishmaniasis"                                                
  [3] "Lipid and atherosclerosis"                                    
  [4] "TNF signaling pathway"                                        
  [5] "Pertussis"                                                    
  [6] "Epstein-Barr virus infection"                                 
  [7] "Herpes simplex virus 1 infection"                             
  [8] "Cytokine-cytokine receptor interaction"                       
  [9] "MAPK signaling pathway"                                       
 [10] "Kaposi sarcoma-associated herpesvirus infection"              
 [11] "Toxoplasmosis"                                                
 [12] "Complement and coagulation cascades"                          
 [13] "Alcoholic liver disease"                                      
 [14] "Pathogenic Escherichia coli infection"                        
 [15] "Toll-like receptor signaling pathway"                         
 [16] "Phagosome"                                                    
 [17] "Tuberculosis"                                                 
 [18] "AGE-RAGE signaling pathway in diabetic complications"         
 [19] "NF-kappa B signaling pathway"                                 
 [20] "Legionellosis"                                                
 [21] "Human cytomegalovirus infection"                              
 [22] "Influenza A"                                                  
 [23] "Chagas disease"                                               
 [24] "NOD-like receptor signaling pathway"                          
 [25] "Malaria"                                                      
 [26] "Apoptosis"                                                    
 [27] "IL-17 signaling pathway"                                      
 [28] "Measles"                                                      
 [29] "Coronavirus disease - COVID-19"                               
 [30] "Human T-cell leukemia virus 1 infection"                      
 [31] "Th17 cell differentiation"                                    
 [32] "Hepatitis B"                                                  
 [33] "Salmonella infection"                                         
 [34] "Efferocytosis"                                                
 [35] "PI3K-Akt signaling pathway"                                   
 [36] "Rheumatoid arthritis"                                         
 [37] "Staphylococcus aureus infection"                              
 [38] "Inflammatory bowel disease"                                   
 [39] "Fluid shear stress and atherosclerosis"                       
 [40] "Human immunodeficiency virus 1 infection"                     
 [41] "Antigen processing and presentation"                          
 [42] "Transcriptional misregulation in cancer"                      
 [43] "Amoebiasis"                                                   
 [44] "HIF-1 signaling pathway"                                      
 [45] "Pancreatic cancer"                                            
 [46] "JAK-STAT signaling pathway"                                   
 [47] "Natural killer cell mediated cytotoxicity"                    
 [48] "Hematopoietic cell lineage"                                   
 [49] "TGF-beta signaling pathway"                                   
 [50] "Chronic myeloid leukemia"                                     
 [51] "Cellular senescence"                                          
 [52] "Hippo signaling pathway"                                      
 [53] "Cell adhesion molecules"                                      
 [54] "Focal adhesion"                                               
 [55] "Shigellosis"                                                  
 [56] "C-type lectin receptor signaling pathway"                     
 [57] "Fc gamma R-mediated phagocytosis"                             
 [58] "Viral

In [16]:
kegg_plier2_summary  %>%  dplyr::filter(LV == "LV102")

LV,pathway
<chr>,<chr>
LV102,"Complement and coagulation cascades, Ferroptosis, Systemic lupus erythematosus"


In [17]:
kegg_list$LV12@result  %>% 
dplyr::filter(p.adjust < 0.05)  %>% 
dplyr::pull(Description) 

[1] "Endocytosis"                                                            
  [2] "Salmonella infection"                                                   
  [3] "Tuberculosis"                                                           
  [4] "Shigellosis"                                                            
  [5] "Epstein-Barr virus infection"                                           
  [6] "Lysosome"                                                               
  [7] "Human immunodeficiency virus 1 infection"                               
  [8] "Yersinia infection"                                                     
  [9] "Kaposi sarcoma-associated herpesvirus infection"                        
 [10] "Human cytomegalovirus infection"                                        
 [11] "Osteoclast differentiation"                                             
 [12] "NOD-like receptor signaling pathway"                                    
 [13] "Phagosome"                                                              
 [14] "Pathogenic Escherichia coli infection"                                  
 [15] "Fc gamma R-mediated phagocytosis"                                       
 [16] "Measles"                                                                
 [17] "Chemokine signaling pathway"                                            
 [18] "C-type lectin receptor signaling pathway"                               
 [19] "PD-L1 expression and PD-1 checkpoint pathway in cancer"                 
 [20] "B cell receptor signaling pathway"                                      
 [21] "Toll-like receptor signaling pathway"                                   
 [22] "Lipid and atherosclerosis"                                              
 [23] "Herpes simplex virus 1 infection"                                       
 [24] "Hepatitis B"                                                            
 [25] "Influenza A"                                                            
 [26] "Viral carcinogenesis"                                                   
 [27] "Epithelial cell signaling in Helicobacter pylori infection"             
 [28] "Apoptosis"                                                              
 [29] "TNF signaling pathway"                                                  
 [30] "Autophagy - animal"                                                     
 [31] "Chronic myeloid leukemia"                                               
 [32] "Neurotrophin signaling pathway"                                         
 [33] "Sphingolipid signaling pathway"                                         
 [34] "Toxoplasmosis"                                                          
 [35] "Necroptosis"                                                            
 [36] "Acute myeloid leukemia"                                                 
 [37] "Leishmaniasis"                                                          
 [38] "Platelet activation"                                                    
 [39] "NF-kappa B signaling pathway"                                           
 [40] "Human T-cell leukemia virus 1 infection"                                
 [41] "Hepatitis C"                                                            
 [42] "Chagas disease"                                                         
 [43] "Fc epsilon RI signaling pathway"                                        
 [44] "Pancreatic cancer"                                                      
 [45] "Pathways of neurodegeneration - multiple diseases"                      
 [46] "Natural killer cell mediated cytotoxicity"                              
 [47] "Human papillomavirus infection"                                         
 [48] "Legionellosis"                                                          
 [49] "Vibrio cholerae infection"                                              
 [50] "Th17 cell differentiation"                                              
 [

In [18]:
gtex_plier2_kegg$summary %>%
dplyr::filter(LV == "LV23") %>% 
dplyr::filter(FDR < 0.05)

,pathway,LV,AUC,p_value,FDR,npos,nneg
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>
KEGG_2021_Human28,Asthma,LV23,0.8255552,7.509568e-03,3.255711e-02,4,21074
KEGG_2021_Human65,Coronavirus disease,LV23,0.6989122,1.339277e-06,4.777623e-05,40,21074
KEGG_2021_Human241,RNA transport,LV23,0.6661013,2.786581e-04,2.814988e-03,31,21074
KEGG_2021_Human254,Ribosome,LV23,0.8109177,3.259455e-09,3.867120e-07,25,21074
KEGG_2021_Human267,Spliceosome,LV23,0.7651025,2.263222e-07,1.148288e-05,26,21074


In [19]:
kegg_list$LV23@result  %>% 
dplyr::filter(p.adjust < 0.05)  %>% 
dplyr::pull(Description) 

kegg_plier2_summary  %>%  dplyr::filter(LV == "LV23")

[1] "Coronavirus disease - COVID-19"                   
 [2] "Non-alcoholic fatty liver disease"                
 [3] "Protein processing in endoplasmic reticulum"      
 [4] "Parkinson disease"                                
 [5] "Huntington disease"                               
 [6] "Chemical carcinogenesis - reactive oxygen species"
 [7] "Mitophagy - animal"                               
 [8] "Diabetic cardiomyopathy"                          
 [9] "Pentose phosphate pathway"                        
[10] "Ferroptosis"                                      
[11] "Oxidative phosphorylation"                        
[12] "DNA replication"                                  
[13] "Carbon metabolism"                                
[14] "Nucleotide metabolism"                            
[15] "Thyroid cancer"                                   
[16] "Prion disease"                                    
[17] "Protein export"                                   
[18] "Thermogenesis"                                    
[19] "Alzheimer disease"                                
[20] "Base excision repair"                             
[21] "Biosynthesis of amino acids"                      
[22] "Amyotrophic lateral sclerosis"                    
[23] "Legionellosis"

LV,pathway
<chr>,<chr>
LV23,"Asthma, Ribosome, Spliceosome"


In [20]:
kegg_list$LV44@result  %>% 
dplyr::filter(p.adjust < 0.05)  %>% 
dplyr::pull(Description) 

kegg_plier2_summary  %>%  dplyr::filter(LV == "LV44")

[1] "Fatty acid metabolism"                                   
 [2] "Carbon metabolism"                                       
 [3] "Fatty acid degradation"                                  
 [4] "Glycine, serine and threonine metabolism"                
 [5] "Biosynthesis of unsaturated fatty acids"                 
 [6] "PPAR signaling pathway"                                  
 [7] "Arginine and proline metabolism"                         
 [8] "Hippo signaling pathway"                                 
 [9] "Biosynthesis of amino acids"                             
[10] "Valine, leucine and isoleucine degradation"              
[11] "Propanoate metabolism"                                   
[12] "Peroxisome"                                              
[13] "One carbon pool by folate"                               
[14] "Notch signaling pathway"                                 
[15] "Thyroid hormone signaling pathway"                       
[16] "Axon guidance"                                           
[17] "beta-Alanine metabolism"                                 
[18] "Basal cell carcinoma"                                    
[19] "Proteoglycans in cancer"                                 
[20] "Wnt signaling pathway"                                   
[21] "Fatty acid elongation"                                   
[22] "Alanine, aspartate and glutamate metabolism"             
[23] "Hedgehog signaling pathway"                              
[24] "Alcoholic liver disease"                                 
[25] "Neuroactive ligand signaling"                            
[26] "Calcium signaling pathway"                               
[27] "Dopaminergic synapse"                                    
[28] "Sphingolipid signaling pathway"                          
[29] "Cysteine and methionine metabolism"                      
[30] "Cytoskeleton in muscle cells"                            
[31] "Human papillomavirus infection"                          
[32] "Efferocytosis"                                           
[33] "Nitrogen metabolism"                                     
[34] "TGF-beta signaling pathway"                              
[35] "GABAergic synapse"                                       
[36] "Signaling pathways regulating pluripotency of stem cells"
[37] "Pyruvate metabolism"                                     
[38] "Glycerophospholipid metabolism"                          
[39] "Lysine degradation"                                      
[40] "Gap junction"                                            
[41] "Rap1 signaling pathway"                                  
[42] "Central carbon metabolism in cancer"                     
[43] "Mineral absorption"                                      
[44] "Glycolysis / Gluconeogenesis"                            
[45] "Glutamatergic synapse"                                   
[46] "Choline metabolism in cancer"                            
[47] "Ferroptosis"                                             
[48] "Hepatocellular carcinoma"                                
[49] "Regulation of actin cytoskeleton"                        
[50] "Long-term depression"                                    
[51] "Proximal tubule bicarbonate reclamation"                 
[52] "2-Oxocarboxylic acid metabolism"                         
[53] "Chagas disease"                                          
[54] "Breast cancer"                                           
[55] "Circadian entrainment"                                   
[56] "Platinum drug resistance"                                
[57] "Melanogenesis"                                           
[58] "Glutathione metabolism"                                  
[59] "Pentose phosphate pathway"                               
[60] "Tryptophan metabolism"                                   
[61] "Focal adhesion"                                          
[62] "Inositol phosphate metabolism"

LV,pathway
<chr>,<chr>
LV44,"Fatty acid degradation, Glycine, serine and threonine metabolism"


# GTEx tissues

## ORA

In [21]:
# Download & parse Enrichr GTEx tissues 
enrichr_url <- "https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GTEx_Tissues_V8_2023"
txt <- content(GET(enrichr_url), as = "text", encoding = "UTF-8")
lines <- strsplit(txt, "\n", fixed = TRUE)[[1]]
parts <- strsplit(lines, "\t", fixed = TRUE)

terms <- vapply(parts, function(x) if (length(x)) x[[1]] else NA_character_, character(1))
genes_list <- lapply(parts, function(x) {
  if (length(x) <= 2) x[2:length(x)] else x[3:length(x)]  
})

GTEx_TERM2GENE_SYM <- tibble(
  term = rep(terms, lengths(genes_list)),
  SYMBOL = unlist(genes_list, use.names = FALSE)
) %>%
  filter(!is.na(term), !is.na(SYMBOL), SYMBOL != "")

sym2eg <- AnnotationDbi::select(
  org.Hs.eg.db,
  keys = unique(GTEx_TERM2GENE_SYM$SYMBOL),
  columns = "ENTREZID",
  keytype = "SYMBOL"
) %>% distinct(SYMBOL, ENTREZID) %>% filter(!is.na(ENTREZID))

GTEx_TERM2GENE <- GTEx_TERM2GENE_SYM %>%
  inner_join(sym2eg, by = "SYMBOL") %>%
  transmute(term, gene = ENTREZID)


# Run ORA for each LV gene set against GTEx 
gtex_list <- lapply(top_entrez, function(ids) {
  if (!length(ids)) return(NULL)
  enricher(
    gene           = ids,
    TERM2GENE      = GTEx_TERM2GENE,
    pvalueCutoff   = 0.05,
    pAdjustMethod  = "BH",
  )
})

'select()' returned 1:many mapping between keys and columns

Warning message in inner_join(., sym2eg, by = "SYMBOL"):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 4590 of `x` matches multiple rows in `y`.
ℹ Row 253 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”


In [22]:
gtex_meta <- read.table(
    here('data/gtex/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt'), 
    sep = '\t', 
    header = TRUE,
    quote = "", 
    fill = TRUE,
    comment.char = "" 
)

Let's use GTEx metadata to see which approach (PLIERfull or PLIERbase + ORA) can better capture the tissue-specific

In [23]:
head(gtex_meta)

,SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,⋯,SME1ANTI,SMSPLTRD,SMBSMMRT,SME1SNSE,SME1PCTS,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS
,<chr>,<int>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<int>,<dbl>
1,GTEX-1117F-0003-SM-58Q7G,NA,B1,,NA,Blood,Whole Blood,0013756,1188,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,GTEX-1117F-0003-SM-5DWSB,NA,B1,,NA,Blood,Whole Blood,0013756,1188,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,GTEX-1117F-0003-SM-6WBT7,NA,B1,,NA,Blood,Whole Blood,0013756,1188,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,GTEX-1117F-0011-R10a-SM-AHZ7F,NA,"B1, A1",,NA,Brain,Brain - Frontal Cortex (BA9),0009834,1193,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,GTEX-1117F-0011-R10b-SM-CYKQ8,NA,"B1, A1",,7.2,Brain,Brain - Frontal Cortex (BA9),0009834,1193,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,GTEX-1117F-0226-SM-5GZZ7,0,B1,"2 pieces, ~15% vessel stroma, rep delineated",6.8,Adipose Tissue,Adipose - Subcutaneous,0002190,1214,1125,⋯,14648800,11999300,0.00315785,14669500,50.0354,0.00310538,0.99474,NA,0,50.1944


In [24]:
gtex_meta_sub <- gtex_meta  %>% 
dplyr::select(SAMPID, SMTS, SMTSD) %>% 
    dplyr::rename(sample =SAMPID, tissue = SMTS, tissue_detail = SMTSD)

In [25]:
gtex_samples <- readRDS(here("output/gtex/gtex_samples.rds"))

In [26]:
gtex_baseRes_B  <- data.frame(gtex_baseRes$B)

In [27]:
colnames(gtex_baseRes_B) <- gtex_samples

In [28]:
rownames(gtex_baseRes_B) <- paste0('LV', seq_len(nrow(gtex_baseRes_B)))

In [29]:
lv1_df <- gtex_baseRes_B %>%
  tibble::rownames_to_column("LV") %>%
  filter(LV == "LV1") %>%
  tidyr::pivot_longer(-LV, names_to = "sample", values_to = "score") %>%
  arrange(desc(score))

In [30]:
lv1_df %>% 
dplyr::left_join(gtex_meta_sub)  %>% 
head(50)

Joining with `by = join_by(sample)`


LV,sample,score,tissue,tissue_detail
<chr>,<chr>,<dbl>,<chr>,<chr>
LV1,GTEX-1399U-2726-SM-9YFKV,0.9121700,Blood Vessel,Artery - Tibial
LV1,GTEX-1CB4F-0726-SM-793AO,0.8942443,Blood Vessel,Artery - Tibial
LV1,GTEX-12WSK-1926-SM-5LZVK,0.8838900,Ovary,Ovary
LV1,GTEX-15UF6-2026-SM-6LPI3,0.8836601,Ovary,Ovary
LV1,GTEX-1GF9W-0226-SM-7PC2U,0.8808621,Blood Vessel,Artery - Tibial
LV1,GTEX-SN8G-2426-SM-EVR31,0.8723529,Ovary,Ovary
LV1,GTEX-17HII-0626-SM-7LT9H,0.8706934,Blood Vessel,Artery - Tibial
LV1,GTEX-12WSK-2726-SM-5CVND,0.8695895,Blood Vessel,Artery - Tibial
LV1,GTEX-132Q8-0011-R11b-SM-5DUW9,0.8661645,Brain,Brain - Cerebellar Hemisphere


In [31]:
gtex_list$LV1@result %>%
  filter(p.adjust < 0.05) %>%
  pull(Description) %>%
  unique()

[1] "Artery - Tibial Male 30-39 Up"                      
 [2] "Artery - Tibial Male 20-29 Up"                      
 [3] "Artery - Tibial Female 40-49 Up"                    
 [4] "Artery - Tibial Female 50-59 Up"                    
 [5] "Artery - Tibial Female 20-29 Up"                    
 [6] "Artery - Tibial Female 60-69 Up"                    
 [7] "Artery - Tibial Male 40-49 Up"                      
 [8] "Artery - Tibial Female 30-39 Up"                    
 [9] "Artery - Tibial Male 60-69 Up"                      
[10] "Artery - Tibial Female 70-79 Up"                    
[11] "Artery - Tibial Male 50-59 Up"                      
[12] "Artery - Aorta Female 20-29 Up"                     
[13] "Artery - Aorta Female 30-39 Up"                     
[14] "Artery - Aorta Male 30-39 Up"                       
[15] "Artery - Coronary Male 70-79 Up"                    
[16] "Artery - Aorta Male 40-49 Up"                       
[17] "Artery - Aorta Female 60-69 Up"                     
[18] "Artery - Aorta Male 20-29 Up"                       
[19] "Artery - Aorta Male 50-59 Up"                       
[20] "Artery - Aorta Female 40-49 Up"                     
[21] "Artery - Aorta Male 70-79 Up"                       
[22] "Artery - Aorta Female 50-59 Up"                     
[23] "Artery - Coronary Female 40-49 Up"                  
[24] "Artery - Aorta Male 60-69 Up"                       
[25] "Artery - Aorta Female 70-79 Up"                     
[26] "Artery - Coronary Male 30-39 Up"                    
[27] "Ovary Female 70-79 Up"                              
[28] "Ovary Female 30-39 Up"                              
[29] "Artery - Tibial Male 70-79 Up"                      
[30] "Esophagus - Gastroesophageal Junction Male 70-79 Up"
[31] "Artery - Coronary Female 50-59 Up"                  
[32] "Artery - Coronary Female 60-69 Up"                  
[33] "Ovary Female 40-49 Up"                              
[34] "Ovary Female 60-69 Up"                              
[35] "Colon - Sigmoid Female 40-49 Up"                    
[36] "Artery - Coronary Female 20-29 Up"                  
[37] "Artery - Coronary Male 50-59 Up"                    
[38] "Artery - Coronary Male 40-49 Up"                    
[39] "Colon - Sigmoid Male 30-39 Up"                      
[40] "Colon - Sigmoid Male 50-59 Up"                      
[41] "Ovary Female 20-29 Up"                              
[42] "Artery - Coronary Male 60-69 Up"                    
[43] "Bladder Male 50-59 Up"                              
[44] "Colon - Sigmoid Female 20-29 Up"                    
[45] "Colon - Sigmoid Male 40-49 Up"                      
[46] "Ovary Female 50-59 Up"                              
[47] "Adrenal Gland Male 30-39 Up"                        
[48] "Colon - Sigmoid Female 50-59 Up"                    
[49] "Colon - Sigmoid Female 60-69 Up"                    
[50] "Colon - Sigmoid Male 20-29 Up"                      
[51] "Esophagus - Muscularis Female 70-79 Up"             
[52] "Uterus Female 20-29 Up"

In [32]:
gtex_plier2_gtex <- readRDS(file.path(output_data_dir, "gtex_GTEx_Tissues_V8_2023_PLIER2.rds"))

In [33]:
gtex_plier2_gtex_summary <- gtex_plier2_gtex$summary %>%
    dplyr::filter(FDR < 0.05 & AUC > 0.7) %>%
    dplyr::group_by(LV) %>%
    dplyr::summarise(pathway = paste(pathway, collapse = ', '), .groups = "drop") %>%
    dplyr::arrange(as.numeric(gsub("[^0-9]", "", LV)))

In [34]:
gtex_plier2_gtex_summary  %>% 
dplyr::filter(LV == "LV1")  %>% 
dplyr::pull(pathway)  %>%
  unique()

[1] "Heart - Atrial Appendage Female 60-69 Up, Heart - Left Ventricle Male 50-59 Up, Artery - Aorta Female 20-29 Up, Colon - Sigmoid Male 30-39 Up"

In [35]:
lv15_df <- gtex_baseRes_B %>%
    tibble::rownames_to_column("LV") %>%
    filter(LV == "LV15") %>%
    tidyr::pivot_longer(-LV, names_to = "sample", values_to = "score") %>%
    arrange(desc(score)) %>%
    dplyr::left_join(gtex_meta_sub)

top_n <- ceiling(0.05 * nrow(lv15_df))
lv15_df %>% slice_head(n = top_n) %>% 
pull(tissue) %>%
table()

lv15_df %>% 
    slice_head(n = top_n) %>% 
    count(tissue) %>% 
    mutate(percent = 100 * n / sum(n))

lv15_df %>% 
    slice_head(n = top_n) %>% 
    count(tissue) %>% 
    arrange(desc(n)) %>% 
    mutate(percent = 100 * as.numeric(n) / sum(as.numeric(n))) %>% 
    mutate(msg = paste0(tissue, ": ", round(percent, 1), "%")) %>% 
    { paste(.$msg, collapse = "; ") }

Joining with `by = join_by(sample)`


.
 Adipose Tissue   Adrenal Gland           Blood    Blood Vessel          Breast 
             21             152              12               4              12 
          Colon       Esophagus           Heart          Kidney           Liver 
             58               2               1              66             226 
          Nerve           Ovary        Pancreas        Prostate            Skin 
              4              16             112              31               2 
Small Intestine          Spleen         Stomach         Thyroid          Uterus 
             63               1              79               6               1 
         Vagina 
              1 

tissue,n,percent
<chr>,<int>,<dbl>
Adipose Tissue,21,2.4137931
Adrenal Gland,152,17.4712644
Blood,12,1.3793103
Blood Vessel,4,0.4597701
Breast,12,1.3793103
Colon,58,6.6666667
Esophagus,2,0.2298851
Heart,1,0.1149425
Kidney,66,7.5862069


[1] "Liver: 26%; Adrenal Gland: 17.5%; Pancreas: 12.9%; Stomach: 9.1%; Kidney: 7.6%; Small Intestine: 7.2%; Colon: 6.7%; Prostate: 3.6%; Adipose Tissue: 2.4%; Ovary: 1.8%; Blood: 1.4%; Breast: 1.4%; Thyroid: 0.7%; Blood Vessel: 0.5%; Nerve: 0.5%; Esophagus: 0.2%; Skin: 0.2%; Heart: 0.1%; Spleen: 0.1%; Uterus: 0.1%; Vagina: 0.1%"

In [36]:
gtex_list$LV15@result %>%
  filter(p.adjust < 0.05) %>%
  pull(Description) %>%
  unique()


tissues <- gtex_list$LV15@result %>%
  dplyr::filter(p.adjust < 0.05) %>%
  dplyr::pull(Description) %>%
  stringr::str_replace("\\s+-\\s+.*$", "") %>%          
  stringr::str_replace("\\s+(Male|Female)\\b.*$", "") %>% 
  stringr::str_replace("\\s+Up$", "") %>%                
  stringr::str_squish() %>%
  unique() %>%
  sort()

paste(tissues, collapse = ", ")

[1] "Liver Male 20-29 Up"                         
 [2] "Liver Female 60-69 Up"                       
 [3] "Liver Female 20-29 Up"                       
 [4] "Liver Male 50-59 Up"                         
 [5] "Liver Male 40-49 Up"                         
 [6] "Liver Male 70-79 Up"                         
 [7] "Liver Female 40-49 Up"                       
 [8] "Liver Female 50-59 Up"                       
 [9] "Liver Male 30-39 Up"                         
[10] "Liver Male 60-69 Up"                         
[11] "Liver Female 30-39 Up"                       
[12] "Adrenal Gland Male 70-79 Up"                 
[13] "Adrenal Gland Female 30-39 Up"               
[14] "Adrenal Gland Female 50-59 Up"               
[15] "Adrenal Gland Male 40-49 Up"                 
[16] "Adrenal Gland Female 20-29 Up"               
[17] "Adrenal Gland Male 50-59 Up"                 
[18] "Adrenal Gland Male 60-69 Up"                 
[19] "Adrenal Gland Female 40-49 Up"               
[20] "Adipose - Subcutaneous Female 20-29 Up"      
[21] "Adipose - Visceral (Omentum) Female 20-29 Up"
[22] "Adrenal Gland Female 60-69 Up"               
[23] "Adipose - Subcutaneous Male 20-29 Up"        
[24] "Adrenal Gland Male 20-29 Up"                 
[25] "Adrenal Gland Male 30-39 Up"                 
[26] "Breast - Mammary Tissue Male 20-29 Up"       
[27] "Adipose - Subcutaneous Female 30-39 Up"      
[28] "Adipose - Subcutaneous Female 70-79 Up"      
[29] "Adipose - Subcutaneous Female 40-49 Up"      
[30] "Kidney - Cortex Male 40-49 Up"               
[31] "Breast - Mammary Tissue Female 70-79 Up"     
[32] "Adipose - Subcutaneous Male 70-79 Up"        
[33] "Adipose - Visceral (Omentum) Female 70-79 Up"
[34] "Breast - Mammary Tissue Male 30-39 Up"

[1] "Adipose, Adrenal Gland, Breast, Kidney, Liver"

In [37]:
gtex_plier2_gtex_summary  %>% 
dplyr::filter(LV == "LV15")  %>% 
dplyr::pull(pathway)  %>%
  unique()

  lv15_df %>%
    slice_head(n = top_n) %>%
    pull(tissue) %>%
    sub(" -.*", "", .) %>%
    unique() %>%
    paste(collapse = "; ") %>%
    paste0("Top tissues: ", .)

[1] "Liver Male 60-69 Up, Liver Male 30-39 Up, Liver Female 60-69 Up, Liver Male 20-29 Up, Adrenal Gland Female 30-39 Up, Liver Male 40-49 Up, Liver Male 70-79 Up, Liver Female 30-39 Up"

[1] "Top tissues: Liver; Spleen; Blood; Kidney; Adipose Tissue; Breast; Heart; Small Intestine; Pancreas; Stomach; Adrenal Gland; Thyroid; Blood Vessel; Uterus; Skin; Colon; Prostate; Ovary; Esophagus; Nerve; Vagina"

In [38]:
kegg_list$LV15@result  %>% 
dplyr::filter(p.adjust < 0.05)  %>% 
dplyr::pull(Description) 

[1] "Peroxisome"                                  
 [2] "Fatty acid degradation"                      
 [3] "Valine, leucine and isoleucine degradation"  
 [4] "Biosynthesis of cofactors"                   
 [5] "Carbon metabolism"                           
 [6] "Fatty acid metabolism"                       
 [7] "Glycine, serine and threonine metabolism"    
 [8] "PPAR signaling pathway"                      
 [9] "Alcoholic liver disease"                     
[10] "Tryptophan metabolism"                       
[11] "One carbon pool by folate"                   
[12] "Pyruvate metabolism"                         
[13] "beta-Alanine metabolism"                     
[14] "Complement and coagulation cascades"         
[15] "Biosynthesis of unsaturated fatty acids"     
[16] "Butanoate metabolism"                        
[17] "Propanoate metabolism"                       
[18] "Drug metabolism - cytochrome P450"           
[19] "Metabolism of xenobiotics by cytochrome P450"
[20] "Retinol metabolism"                          
[21] "Glyoxylate and dicarboxylate metabolism"     
[22] "Histidine metabolism"                        
[23] "Tyrosine metabolism"                         
[24] "Folate transport and metabolism"             
[25] "Pantothenate and CoA biosynthesis"           
[26] "Folate biosynthesis"                         
[27] "Biosynthesis of amino acids"                 
[28] "Glycolysis / Gluconeogenesis"                
[29] "Vitamin digestion and absorption"            
[30] "Arginine and proline metabolism"             
[31] "Nicotinate and nicotinamide metabolism"      
[32] "Cysteine and methionine metabolism"          
[33] "Chemical carcinogenesis - DNA adducts"       
[34] "Alanine, aspartate and glutamate metabolism" 
[35] "Glutathione metabolism"                      
[36] "Cholesterol metabolism"                      
[37] "Arginine biosynthesis"                       
[38] "Fatty acid elongation"                       
[39] "Selenocompound metabolism"                   
[40] "2-Oxocarboxylic acid metabolism"             
[41] "Ferroptosis"                                 
[42] "Drug metabolism - other enzymes"             
[43] "Citrate cycle (TCA cycle)"                   
[44] "Lysine degradation"                          
[45] "Terpenoid backbone biosynthesis"             
[46] "Glycerolipid metabolism"                     
[47] "Steroid biosynthesis"                        
[48] "Lipoic acid metabolism"                      
[49] "Biosynthesis of nucleotide sugars"           
[50] "Primary bile acid biosynthesis"              
[51] "Sulfur metabolism"                           
[52] "Insulin resistance"                          
[53] "Pentose phosphate pathway"                   
[54] "Amino sugar and nucleotide sugar metabolism" 
[55] "Platinum drug resistance"                    
[56] "Antifolate resistance"                       
[57] "ABC transporters"                            
[58] "Pertussis"                                   
[59] "Glucagon signaling pathway"                  
[60] "Ascorbate and aldarate metabolism"           
[61] "Glycerophospholipid metabolism"              
[62] "Pyrimidine metabolism"                       
[63] "AMPK signaling pathway"                      
[64] "Proximal tubule bicarbonate reclamation"     
[65] "Insulin signaling pathway"                   
[66] "Mineral absorption"                          
[67] "Nucleotide metabolism"

In [39]:
kegg_plier2_summary  %>%  dplyr::filter(LV == "LV15")

LV,pathway
<chr>,<chr>
LV15,"Drug metabolism, Glycine, serine and threonine metabolism, Peroxisome, Retinol metabolism"
